In [ ]:
import os
import sys
import json
from src.ts_main import run_ts, parse_input_dict

In [ ]:
TS_BASE_DIR = ".."

# Example 1

In [ ]:
quinazoline_json = """{
"reagent_file_list": [
        "TS_BASE_DIR/data/aminobenzoic_100.smi",
        "TS_BASE_DIR/data/primary_amines_100.smi",
        "TS_BASE_DIR/data/carboxylic_acids_100.smi"
    ],
    "reaction_smarts": "N[c:4][c:3]C(O)=O.[#6:1][NH2].[#6:2]C(=O)[OH]>>[C:2]c1n[c:4][c:3]c(=O)n1[C:1]",
    "num_warmup_trials": 10,
    "num_ts_iterations": 5000,
    "evaluator_class_name": "FPEvaluator",
    "evaluator_arg": {"query_smiles" : "COc1cccc2c(=O)n([C@@H]3C[C@@H](O)[C@@H](O)C3)c([C@@H](N)CNC(N)=O)nc12"},
    "ts_mode": "maximize",
    "log_filename": "ts_logs.txt",
    "results_filename": "ts_results.csv"
}""".replace("TS_BASE_DIR", TS_BASE_DIR)

In [ ]:
quinazoline_dict = json.loads(quinazoline_json)
parse_input_dict(quinazoline_dict)

In [ ]:
run_ts(quinazoline_dict, inline_log=True, save_output=False)

# Example 2

In [ ]:
custom_json = """{
"reagent_file_list": [
        "TS_BASE_DIR/data/data_ric/ald.smi",
        "TS_BASE_DIR/data/data_ric/ammines.smi"
    ],
    "reaction_smarts": "[#6:1]-[#6H1:4]=O.[#6:3]-[#7H2:2]>>[#6:3]-[#7:2]-[#6]-1=[#6](-[#6]=[#6]-[#6]-2=[#7]-[#6](-[#6]-[#7]-3-[#6]-[#6]-[#6](-[#6]-[#6]-3)-[#7]-3-[#6]=[#7]-[#6]=[#6:4]-3-[#6:1])=[#6]-[#7]-1-2)-[#6]-1=[#7]-[#7]=[#7]-[#7]-1",
    "num_warmup_trials": 10,
    "num_ts_iterations": 5000,
    "evaluator_class_name": "FPEvaluator",
    "evaluator_arg": {"query_smiles" : "OC(=O)c1ccc2nc(CN3CCC(CC3)c3cccc(OCc4ccc(cc4F)C#N)n3)n(C[C@@H]3CCO3)c2c1"},
    "ts_mode": "maximize",
    "log_filename": "ts_logs.txt",
    "results_filename": "ts_results.csv"
}""".replace("TS_BASE_DIR", TS_BASE_DIR)

In [ ]:
custom_dict = json.loads(custom_json)
parse_input_dict(custom_dict)

In [ ]:
run_ts(custom_dict, inline_log=True, save_output=False)